In [28]:
import pandas as pd
import numpy as np
import nltk
import re
sent_token = nltk.sent_tokenize
import csv  
from nltk import sent_tokenize, word_tokenize, pos_tag
import re
from sklearn.feature_extraction.text import CountVectorizer
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

file_path = '/root/Springboard/Data/cleaned_article_data.csv'
clean_art2 = pd.read_csv(file_path, index_col = False)
clean_art2

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,by stephen hiltner and susan lehman,article,finding an expansive view of a forgotten peop...,photography new york times niger ferguson adam...,3,insider,2,20170401 001541,unknown,one of the largest photo displays in times his...,the new york times,news,httpswwwnytimescom20170331insidernigermigrants...
1,NaN,58def3237c459f24986d7c84,823,by gail collins,article,and now the dreaded trump curse,united states politics and government trump do...,3,oped,23,20170401 002358,unknown,meet the gang from under the bus,the new york times,oped,httpswwwnytimescom20170331opinionandnowthedrea...
2,NaN,58def9f57c459f24986d7c90,575,by the editorial board,article,venezuelas descent into dictatorship,venezuela politics and government maduro nicolas,3,editorial,22,20170401 005306,unknown,a court ruling annulling the legislatures auth...,the new york times,editorial,httpswwwnytimescom20170331opinionvenezuelasdes...
3,NaN,58defd317c459f24986d7c95,1374,by michael powell,article,stain permeates basketball blue blood,basketball college university of north carolin...,3,sports,1,20170401 010652,college basketball,for two decades until 2013 north carolina enga...,the new york times,news,httpswwwnytimescom20170331sportsncaabasketball...


In [29]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(clean_art2)
cv_matrix = cv_matrix.toarray()
cv_matrix

vocab = cv.get_feature_names()
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

bv = CountVectorizer(ngram_range=(2,2))
#bv_matrix = bv.fit_transform(clean_art2)

#bv_matrix = bv_matrix.toarray()
#vocab = bv.get_feature_names()
#pd.DataFrame(bv_matrix, columns=vocab)

from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(clean_art2)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
#similarity_df

In [32]:
file_path = '/root/Springboard/Data/cleaned_comment_data.csv'
clean_comments = pd.read_csv(file_path, index_col = False)

for index, column in enumerate(clean_comments):
    clean_comments[column] = clean_comments[column].fillna(0)

clean_comments.head(5)


from sklearn import preprocessing
le = preprocessing.LabelEncoder()


features = clean_comments.columns.tolist()
output = 'recommendations'
features.remove('recommendations')

for column in clean_comments.columns:
    clean_comments[column] = clean_comments[column].astype(str)
    if clean_comments[column].dtype == type(object):
        clean_comments[column] = le.fit_transform(clean_comments[column])

#print(features)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

clean_comments.astype(int)
clean_comments.head(5)

#clean_comments.dtypes

approveDate              int64
commentBody              int64
commentID                int64
commentSequence          int64
commentTitle             int64
commentType              int64
createDate               int64
depth                    int64
editorsSelection         int64
parentID                 int64
parentUserDisplayName    int64
permID                   int64
picURL                   int64
recommendations          int64
recommendedFlag          int64
replyCount               int64
reportAbuseFlag          int64
sharing                  int64
status                   int64
timespeople              int64
trusted                  int64
updateDate               int64
userDisplayName          int64
userID                   int64
userLocation             int64
userTitle                int64
userURL                  int64
inReplyTo                int64
articleID                int64
sectionName              int64
newDesk                  int64
articleWordCount         int64
printPag

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier()

param = {'n_estimators': [5,10,100],
         'max_depth': [10, 30, 100] }

gs=GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(clean_comments[features], clean_comments[output])
pd.DataFrame(gs_fit.cv_results_).head(100)

/root/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/root/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.012265,0.002222,0.001674,0.000266,10,5,"{'max_depth': 10, 'n_estimators': 5}",0.018116,0.063725,0.119565,0.093023,0.048780,0.064,0.036497,9
1,0.035743,0.009296,0.002700,0.000572,10,10,"{'max_depth': 10, 'n_estimators': 10}",0.036232,0.107843,0.146739,0.104651,0.054878,0.086,0.040969,5
2,0.231393,0.034865,0.010233,0.000443,10,100,"{'max_depth': 10, 'n_estimators': 100}",0.032609,0.117647,0.130435,0.127907,0.085366,0.093,0.040094,1
3,0.018620,0.003477,0.001511,0.000153,30,5,"{'max_depth': 30, 'n_estimators': 5}",0.032609,0.088235,0.141304,0.116279,0.097561,0.089,0.039002,2
4,0.036018,0.006170,0.001958,0.000062,30,10,"{'max_depth': 30, 'n_estimators': 10}",0.021739,0.078431,0.152174,0.098837,0.091463,0.082,0.044434,7
5,0.324497,0.060506,0.010231,0.000775,30,100,"{'max_depth': 30, 'n_estimators': 100}",0.021739,0.102941,0.141304,0.122093,0.091463,0.089,0.044509,2
6,0.017787,0.003288,0.001473,0.000201,100,5,"{'max_depth': 100, 'n_estimators': 5}",0.043478,0.068627,0.108696,0.104651,0.079268,0.077,0.025331,8
7,0.034286,0.006247,0.001937,0.000135,100,10,"{'max_depth': 100, 'n_estimators': 10}",0.028986,0.068627,0.125000,0.127907,0.097561,0.083,0.039416,6
8,0.329197,0.059610,0.010740,0.000944,100,100,"{'max_depth': 100, 'n_estimators': 100}",0.025362,0.093137,0.152174,0.127907,0.073171,0.088,0.046433,4
